In [1]:
import csv
import os
import cv2
import pickle
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [2]:
csv_path = "./data/driver_imgs_list.csv"
data_path = "./data/imgs"
train_sub_folder = "train"
diver_class_names = {"c0": "safe driving",
                    "c1": "texting - right",
                    "c2": "talking on the phone - right",
                    "c3": "texting - left",
                    "c4": "talking on the phone - left",
                    "c5": "operating the radio",
                    "c6": "drinking",
                    "c7": "reaching behind",
                    "c8": "hair and makeup",
                    "c9": "talking to passenger"}

In [3]:
train_image = []
with open(csv_path) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter = ',')
    line_cnt = 0
    for row in csv_reader:
        # Skipping header part
        if line_cnt == 0:
            line_cnt += 1
            continue

        train_image.append(row)
        line_cnt += 1

In [4]:
print(len(train_image))

22424


In [ ]:
img = cv2.imread(os.path.join(data_path,train_sub_folder,train_image[1][1],train_image[1][2]))
print(os.path.join(data_path,train_sub_folder,train_image[1][1],train_image[1][2]))
print(img)
# cv2.imshow(diver_class_names[train_image[1][1]],img)
# cv2.waitKey(0)

In [5]:
def generator(samples, batch_size=32):
    num_samples = len(samples)
    print(num_samples)
    #while 1: # Loop forever so the generator never terminates
    shuffle(samples)
    for offset in range(0, num_samples, batch_size):
        batch_samples = samples[offset:offset+batch_size]
        images = []
        labels = []
        for batch_sample in batch_samples:
            #print(batch_sample[1][1])
            current_path = os.path.join(data_path,train_sub_folder,batch_sample[1],batch_sample[2])
            image = cv2.imread(current_path)
            image = cv2.resize(image, (32, 32)) 
            imgs_gray = cv2.cvtColor( image, cv2.COLOR_RGB2GRAY )
            imgs_gray = imgs_gray.reshape((imgs_gray.shape + (1,)))
            images.append(imgs_gray)
            labels.append(int(batch_sample[1][1:]))
        X_train = np.asfarray(images,dtype='float32')
        y_train = np.asfarray(labels)
        #print("X_train:", len(X_train))
        X_train = (X_train - 128) /128
        yield (X_train, y_train)

In [6]:
from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(train_image, test_size=0.2)
print(len(train_samples)+len(validation_samples))

22424


In [7]:
# for x,y in generator(train_samples,3):
#     print (type(x[0][0][0][0]),type(y))
X_train_all =[]
y_train_all = []
X_valid =[]
y_valid =[]
# for i in train_samples:
#     images =[]
#     labels =[]
#     current_path = os.path.join(data_path,train_sub_folder,i[1],i[2])
#     image = cv2.imread(current_path)
#     image = cv2.resize(image, (32, 32)) 
#     imgs_gray = cv2.cvtColor( image, cv2.COLOR_RGB2GRAY )
#     imgs_gray = imgs_gray.reshape((imgs_gray.shape + (1,)))
#     images.append(imgs_gray)
#     labels.append(int(i[1][1:]))
#     X_train = np.asfarray(images,dtype='float32')
#     y_train = np.asfarray(labels)

for i in validation_samples:
    images =[]
    labels =[]
    current_path = os.path.join(data_path,train_sub_folder,i[1],i[2])
    image = cv2.imread(current_path)
    image = cv2.resize(image, (32, 32)) 
    imgs_gray = cv2.cvtColor( image, cv2.COLOR_RGB2GRAY )
    imgs_gray = imgs_gray.reshape((imgs_gray.shape + (1,)))
    images.append(imgs_gray)
    labels.append(int(i[1][1:]))
    X_valid = np.asfarray(images,dtype='float32')
    y_valid = np.asfarray(labels)



In [ ]:
x = "   3481 "
print("dvsds",x.strip(),"fdvev")

In [8]:
n_classes =len(diver_class_names)

In [9]:
from tensorflow.contrib.layers import flatten

def LeNet(x):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    
    # SOLUTION: Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x6.
    conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 1, 6), mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros(6))
    conv1   = tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='VALID') + conv1_b
    
    # SOLUTION: Activation.
    conv1 = tf.nn.relu(conv1)

    # SOLUTION: Pooling. Input = 28x28x6. Output = 14x14x6.
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # SOLUTION: Layer 2: Convolutional. Output = 10x10x16.
    conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 6, 16), mean = mu, stddev = sigma))
    conv2_b = tf.Variable(tf.zeros(16))
    conv2   = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='VALID') + conv2_b
    
    # SOLUTION: Activation.
    conv2 = tf.nn.relu(conv2)

    # SOLUTION: Pooling. Input = 10x10x16. Output = 5x5x16.
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # SOLUTION: Flatten. Input = 5x5x16. Output = 400.
    fc0   = flatten(conv2)
    
    # SOLUTION: Layer 3: Fully Connected. Input = 400. Output = 120.
    fc1_W = tf.Variable(tf.truncated_normal(shape=(400, 120), mean = mu, stddev = sigma))
    fc1_b = tf.Variable(tf.zeros(120))
    fc1   = tf.matmul(fc0, fc1_W) + fc1_b
    
    # SOLUTION: Activation.
    fc1    = tf.nn.relu(fc1)
    fc1    = tf.nn.dropout(fc1, keep_prob)
    # SOLUTION: Layer 4: Fully Connected. Input = 120. Output = 84.
    fc2_W  = tf.Variable(tf.truncated_normal(shape=(120, 84), mean = mu, stddev = sigma))
    fc2_b  = tf.Variable(tf.zeros(84))
    fc2    = tf.matmul(fc1, fc2_W) + fc2_b
    
    # SOLUTION: Activation.
    fc2    = tf.nn.relu(fc2)
    fc2    = tf.nn.dropout(fc2, keep_prob)
    # SOLUTION: Layer 5: Fully Connected. Input = 84. Output = n_classes.
    fc3_W  = tf.Variable(tf.truncated_normal(shape=(84, n_classes), mean = mu, stddev = sigma))
    fc3_b  = tf.Variable(tf.zeros(n_classes))
    logits = tf.matmul(fc2, fc3_W) + fc3_b
    
    return logits

In [ ]:
# from tensorflow.contrib.layers import flatten

# def LeNet(x):    
#     # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
#     mu = 0
#     sigma = 0.1
    
#     # SOLUTION: Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x6.
#     conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 1, 6), mean = mu, stddev = sigma))
#     conv1_b = tf.Variable(tf.zeros(6))
#     conv1   = tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='VALID') + conv1_b
    
#     # SOLUTION: Activation.
#     conv1 = tf.nn.relu(conv1)

#     # SOLUTION: Pooling. Input = 28x28x6. Output = 14x14x6.
#     conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

#     # SOLUTION: Layer 2: Convolutional. Output = 10x10x16.
#     conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 6, 16), mean = mu, stddev = sigma))
#     conv2_b = tf.Variable(tf.zeros(16))
#     conv2   = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='VALID') + conv2_b
    
#     # SOLUTION: Activation.
#     conv2 = tf.nn.relu(conv2)

#     # SOLUTION: Pooling. Input = 10x10x16. Output = 5x5x16.
#     conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

#     # SOLUTION: Flatten. Input = 5x5x16. Output = 400.
#     fc0   = flatten(conv2)
    
#     # SOLUTION: Layer 3: Fully Connected. Input = 400. Output = 120.
#     fc1_W = tf.Variable(tf.truncated_normal(shape=(293904, 150000), mean = mu, stddev = sigma))
#     fc1_b = tf.Variable(tf.zeros(150000))
#     fc1   = tf.matmul(fc0, fc1_W) + fc1_b
    
#     # SOLUTION: Activation.
#     fc1    = tf.nn.relu(fc1)
#     fc1    = tf.nn.dropout(fc1, keep_prob)
    
#      # SOLUTION: Layer 3: Fully Connected. Input = 400. Output = 120.
#     fc2_W = tf.Variable(tf.truncated_normal(shape=(150000, 75000), mean = mu, stddev = sigma))
#     fc2_b = tf.Variable(tf.zeros(75000))
#     fc2   = tf.matmul(fc1, fc2_W) + fc2_b
    
#     # SOLUTION: Activation.
#     fc2    = tf.nn.relu(fc2)
#     fc2    = tf.nn.dropout(fc2, keep_prob)
    
#     # SOLUTION: Layer 3: Fully Connected. Input = 400. Output = 120.
#     fc3_W = tf.Variable(tf.truncated_normal(shape=(75000, 25000), mean = mu, stddev = sigma))
#     fc3_b = tf.Variable(tf.zeros(25000))
#     fc3   = tf.matmul(fc2, fc3_W) + fc3_b
    
#     # SOLUTION: Activation.
#     fc3    = tf.nn.relu(fc3)
#     fc3    = tf.nn.dropout(fc3, keep_prob)
    
#     # SOLUTION: Layer 3: Fully Connected. Input = 400. Output = 120.
#     fc4_W = tf.Variable(tf.truncated_normal(shape=(25000, 5000), mean = mu, stddev = sigma))
#     fc4_b = tf.Variable(tf.zeros(5000))
#     fc4   = tf.matmul(fc3, fc4_W) + fc4_b
    
#     # SOLUTION: Activation.
#     fc4   = tf.nn.relu(fc4)
#     fc4    = tf.nn.dropout(fc4, keep_prob)
    
#     # SOLUTION: Layer 3: Fully Connected. Input = 400. Output = 120.
#     fc5_W = tf.Variable(tf.truncated_normal(shape=(5000, 500), mean = mu, stddev = sigma))
#     fc5_b = tf.Variable(tf.zeros(500))
#     fc5   = tf.matmul(fc4, fc5_W) + fc5_b
    
#     # SOLUTION: Activation.
#     fc5    = tf.nn.relu(fc5)
#     fc5    = tf.nn.dropout(fc5, keep_prob)
    
#         # SOLUTION: Layer 3: Fully Connected. Input = 400. Output = 120.
#     fc6_W = tf.Variable(tf.truncated_normal(shape=(500, 84), mean = mu, stddev = sigma))
#     fc6_b = tf.Variable(tf.zeros(84))
#     fc6   = tf.matmul(fc5, fc6_W) + fc6_b
    
#     # SOLUTION: Activation.
#     fc6    = tf.nn.relu(fc6)
#     fc6    = tf.nn.dropout(fc6, keep_prob)
    
# #     # SOLUTION: Layer 4: Fully Connected. Input = 120. Output = 84.
# #     fc2_W  = tf.Variable(tf.truncated_normal(shape=(84, 84), mean = mu, stddev = sigma))
# #     fc2_b  = tf.Variable(tf.zeros(84))
# #     fc2    = tf.matmul(fc1, fc2_W) + fc2_b
    
# #     # SOLUTION: Activation.
# #     fc2    = tf.nn.relu(fc2)
# #     fc2    = tf.nn.dropout(fc2, keep_prob)
#     # SOLUTION: Layer 5: Fully Connected. Input = 84. Output = n_classes.
#     fc7_W  = tf.Variable(tf.truncated_normal(shape=(84, len(diver_class_names)), mean = mu, stddev = sigma))
#     fc7_b  = tf.Variable(tf.zeros(len(diver_class_names)))
#     logits = tf.matmul(fc6, fc7_W) + fc7_b
    
#     return logits

In [10]:
# Features and Labels
x = tf.placeholder(tf.float32, (None, 32, 32,1)) #3 for RGB images and 1 for Gray Scale image
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, len(diver_class_names))
keep_prob = tf.placeholder(tf.float32)

rate = 0.0009

logits = LeNet(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    total_loss = 0
    sess = tf.get_default_session()
    BATCH_SIZE = 32
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy,loss = sess.run([accuracy_operation,loss_operation], feed_dict={x: batch_x, y: batch_y,keep_prob: 1.0})
        total_accuracy += (accuracy * len(batch_x))
        total_loss += (loss * len(batch_x))
    return total_accuracy / num_examples,total_loss / num_examples


In [11]:
#Training the Model
import time
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #num_examples = len(X_train)
    epoch_train_loss =[]
    epoch_validation_loss =[]
    print("Training...")
    print()
    for i in range(5):
        s_t = time.time()
        count =0
       # X_train, y_train = shuffle(X_train, y_train)
        for batch_x,batch_y in generator(train_samples, batch_size=32):
#             end = offset + BATCH_SIZE
#             batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y,keep_prob: 0.5})
            count+=32
#             print(count)
            train_accuracy,train_loss = evaluate(batch_x, batch_y)
            print("Image count :",count,"train_loss :",train_loss,"Train accuracy:",train_accuracy)
        e_t = time.time()
        print("time",e_t-s_t)
            
        validation_accuracy,validation_loss = evaluate(X_valid, y_valid)
        epoch_validation_loss.append(validation_loss)
#         train_accuracy,train_loss = evaluate(X_train_all, y_train_all)
#         epoch_train_loss.append(train_loss)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy ",validation_accuracy,"Validation loss",validation_loss)
        print("train Accuracy = {:.3f}".format(train_accuracy))
        
#     print(epoch_validation_loss) 
    #print(epoch_train_loss)
    saver.save(sess, './lenet')
    print("Model saved")

Training...

17939
Image count : 32 train_loss : 2.29793167114 Train accuracy: 0.09375
Image count : 64 train_loss : 2.34254455566 Train accuracy: 0.03125
Image count : 96 train_loss : 2.2994325161 Train accuracy: 0.0
Image count : 128 train_loss : 2.33758330345 Train accuracy: 0.03125
Image count : 160 train_loss : 2.31677436829 Train accuracy: 0.0625
Image count : 192 train_loss : 2.29979133606 Train accuracy: 0.03125
Image count : 224 train_loss : 2.31522798538 Train accuracy: 0.03125
Image count : 256 train_loss : 2.31668663025 Train accuracy: 0.0625
Image count : 288 train_loss : 2.28073072433 Train accuracy: 0.15625
Image count : 320 train_loss : 2.29417610168 Train accuracy: 0.15625
Image count : 352 train_loss : 2.29710769653 Train accuracy: 0.09375
Image count : 384 train_loss : 2.30397129059 Train accuracy: 0.03125
Image count : 416 train_loss : 2.29625320435 Train accuracy: 0.15625
Image count : 448 train_loss : 2.29306149483 Train accuracy: 0.0625
Image count : 480 train_lo

Image count : 3840 train_loss : 2.27195549011 Train accuracy: 0.1875
Image count : 3872 train_loss : 2.31595468521 Train accuracy: 0.0
Image count : 3904 train_loss : 2.30642127991 Train accuracy: 0.0625
Image count : 3936 train_loss : 2.29645562172 Train accuracy: 0.09375
Image count : 3968 train_loss : 2.29272651672 Train accuracy: 0.09375
Image count : 4000 train_loss : 2.29960393906 Train accuracy: 0.03125
Image count : 4032 train_loss : 2.29392004013 Train accuracy: 0.125
Image count : 4064 train_loss : 2.31419539452 Train accuracy: 0.0625
Image count : 4096 train_loss : 2.28524708748 Train accuracy: 0.09375
Image count : 4128 train_loss : 2.30746245384 Train accuracy: 0.09375
Image count : 4160 train_loss : 2.30002331734 Train accuracy: 0.0625
Image count : 4192 train_loss : 2.29114723206 Train accuracy: 0.1875
Image count : 4224 train_loss : 2.29166889191 Train accuracy: 0.1875
Image count : 4256 train_loss : 2.30028247833 Train accuracy: 0.15625
Image count : 4288 train_loss : 

Image count : 7648 train_loss : 2.24062728882 Train accuracy: 0.1875
Image count : 7680 train_loss : 2.24457144737 Train accuracy: 0.1875
Image count : 7712 train_loss : 2.23355507851 Train accuracy: 0.21875
Image count : 7744 train_loss : 2.23050403595 Train accuracy: 0.3125
Image count : 7776 train_loss : 2.2384967804 Train accuracy: 0.25
Image count : 7808 train_loss : 2.25955629349 Train accuracy: 0.1875
Image count : 7840 train_loss : 2.23051118851 Train accuracy: 0.28125
Image count : 7872 train_loss : 2.24837517738 Train accuracy: 0.21875
Image count : 7904 train_loss : 2.23276519775 Train accuracy: 0.25
Image count : 7936 train_loss : 2.25854969025 Train accuracy: 0.09375
Image count : 7968 train_loss : 2.25393629074 Train accuracy: 0.28125
Image count : 8000 train_loss : 2.22520399094 Train accuracy: 0.25
Image count : 8032 train_loss : 2.18788909912 Train accuracy: 0.28125
Image count : 8064 train_loss : 2.18843960762 Train accuracy: 0.375
Image count : 8096 train_loss : 2.18

Image count : 11424 train_loss : 2.103972435 Train accuracy: 0.3125
Image count : 11456 train_loss : 1.99148094654 Train accuracy: 0.25
Image count : 11488 train_loss : 2.03876328468 Train accuracy: 0.25
Image count : 11520 train_loss : 1.93967008591 Train accuracy: 0.53125
Image count : 11552 train_loss : 2.0712146759 Train accuracy: 0.28125
Image count : 11584 train_loss : 2.00411438942 Train accuracy: 0.25
Image count : 11616 train_loss : 1.9684548378 Train accuracy: 0.40625
Image count : 11648 train_loss : 2.01210689545 Train accuracy: 0.375
Image count : 11680 train_loss : 2.08890199661 Train accuracy: 0.1875
Image count : 11712 train_loss : 1.97884106636 Train accuracy: 0.375
Image count : 11744 train_loss : 2.04307818413 Train accuracy: 0.25
Image count : 11776 train_loss : 1.94168841839 Train accuracy: 0.375
Image count : 11808 train_loss : 2.01872324944 Train accuracy: 0.3125
Image count : 11840 train_loss : 1.999309659 Train accuracy: 0.28125
Image count : 11872 train_loss : 

Image count : 15200 train_loss : 1.68690443039 Train accuracy: 0.46875
Image count : 15232 train_loss : 1.66451871395 Train accuracy: 0.4375
Image count : 15264 train_loss : 1.49769568443 Train accuracy: 0.46875
Image count : 15296 train_loss : 1.65992164612 Train accuracy: 0.34375
Image count : 15328 train_loss : 1.63471662998 Train accuracy: 0.40625
Image count : 15360 train_loss : 1.79526257515 Train accuracy: 0.375
Image count : 15392 train_loss : 1.56457257271 Train accuracy: 0.53125
Image count : 15424 train_loss : 1.64413452148 Train accuracy: 0.46875
Image count : 15456 train_loss : 1.64909255505 Train accuracy: 0.375
Image count : 15488 train_loss : 1.73908615112 Train accuracy: 0.4375
Image count : 15520 train_loss : 1.54796183109 Train accuracy: 0.65625
Image count : 15552 train_loss : 1.72022747993 Train accuracy: 0.53125
Image count : 15584 train_loss : 1.57658672333 Train accuracy: 0.625
Image count : 15616 train_loss : 1.67870903015 Train accuracy: 0.40625
Image count : 

...

Image count : 3136 train_loss : 0.112177170813 Train accuracy: 0.96875
Image count : 3168 train_loss : 0.15761077404 Train accuracy: 1.0
Image count : 3200 train_loss : 0.101160131395 Train accuracy: 0.96875
Image count : 3232 train_loss : 0.177885368466 Train accuracy: 0.9375
Image count : 3264 train_loss : 0.180862575769 Train accuracy: 0.9375
Image count : 3296 train_loss : 0.173604443669 Train accuracy: 0.9375
Image count : 3328 train_loss : 0.16893196106 Train accuracy: 0.90625
Image count : 3360 train_loss : 0.29611441493 Train accuracy: 0.875
Image count : 3392 train_loss : 0.280319392681 Train accuracy: 0.90625
Image count : 3424 train_loss : 0.0518558099866 Train accuracy: 1.0
Image count : 3456 train_loss : 0.399393677711 Train accuracy: 0.875
Image count : 3488 train_loss : 0.188359469175 Train accuracy: 0.96875
Image count : 3520 train_loss : 0.200599655509 Train accuracy: 0.90625
Image count : 3552 train_loss : 0.1007020697 Train accuracy: 0.96875
Image count : 3584 train_

Image count : 6880 train_loss : 0.227418094873 Train accuracy: 0.90625
Image count : 6912 train_loss : 0.108229875565 Train accuracy: 1.0
Image count : 6944 train_loss : 0.296995490789 Train accuracy: 0.9375
Image count : 6976 train_loss : 0.232741653919 Train accuracy: 0.90625
Image count : 7008 train_loss : 0.123821720481 Train accuracy: 0.96875
Image count : 7040 train_loss : 0.452693611383 Train accuracy: 0.875
Image count : 7072 train_loss : 0.207904934883 Train accuracy: 0.9375
Image count : 7104 train_loss : 0.306193739176 Train accuracy: 0.90625
Image count : 7136 train_loss : 0.391226768494 Train accuracy: 0.84375
Image count : 7168 train_loss : 0.250923365355 Train accuracy: 0.90625
Image count : 7200 train_loss : 0.232371285558 Train accuracy: 0.90625
Image count : 7232 train_loss : 0.46646669507 Train accuracy: 0.9375
Image count : 7264 train_loss : 0.297048389912 Train accuracy: 0.90625
Image count : 7296 train_loss : 0.262885808945 Train accuracy: 0.9375
Image count : 732

Image count : 10624 train_loss : 0.157975643873 Train accuracy: 0.96875
Image count : 10656 train_loss : 0.0808038562536 Train accuracy: 1.0
Image count : 10688 train_loss : 0.133578076959 Train accuracy: 0.96875
Image count : 10720 train_loss : 0.338845729828 Train accuracy: 0.875
Image count : 10752 train_loss : 0.134140461683 Train accuracy: 0.9375
Image count : 10784 train_loss : 0.165718972683 Train accuracy: 0.9375
Image count : 10816 train_loss : 0.218328744173 Train accuracy: 0.9375
Image count : 10848 train_loss : 0.0582562610507 Train accuracy: 0.96875
Image count : 10880 train_loss : 0.0664582550526 Train accuracy: 1.0
Image count : 10912 train_loss : 0.101469159126 Train accuracy: 1.0
Image count : 10944 train_loss : 0.226186424494 Train accuracy: 0.90625
Image count : 10976 train_loss : 0.151736587286 Train accuracy: 1.0
Image count : 11008 train_loss : 0.0726813822985 Train accuracy: 1.0
Image count : 11040 train_loss : 0.0888640731573 Train accuracy: 1.0
Image count : 11

Image count : 14336 train_loss : 0.286725193262 Train accuracy: 0.90625
Image count : 14368 train_loss : 0.140170037746 Train accuracy: 0.9375
Image count : 14400 train_loss : 0.157791435719 Train accuracy: 0.96875
Image count : 14432 train_loss : 0.221554219723 Train accuracy: 0.9375
Image count : 14464 train_loss : 0.294095039368 Train accuracy: 0.84375
Image count : 14496 train_loss : 0.101291716099 Train accuracy: 1.0
Image count : 14528 train_loss : 0.237596809864 Train accuracy: 0.9375
Image count : 14560 train_loss : 0.154529094696 Train accuracy: 0.96875
Image count : 14592 train_loss : 0.177581503987 Train accuracy: 0.9375
Image count : 14624 train_loss : 0.319548726082 Train accuracy: 0.875
Image count : 14656 train_loss : 0.381424188614 Train accuracy: 0.90625
Image count : 14688 train_loss : 0.227783590555 Train accuracy: 0.9375
Image count : 14720 train_loss : 0.134739369154 Train accuracy: 0.96875
Image count : 14752 train_loss : 0.135587662458 Train accuracy: 0.96875
Ima

Model saved
